In [1]:
library(cluster)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
#https://www.kaggle.com/alopez247/pokemon
raw.df <- read.csv("../data/raw/pokemon/pokemon_alopez247.csv")
head(raw.df)

Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
4,Charmander,Fire,,309,39,52,43,60,50,...,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
5,Charmeleon,Fire,,405,58,64,58,80,65,...,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed
6,Charizard,Fire,Flying,534,78,84,78,109,85,...,Red,True,0.875,Monster,Dragon,True,1.70,90.5,45,bipedal_tailed


In [3]:
fill_type2 <- function(row){
    if (row["Type_2"] == ''){
        return(row["Type_1"])
    } else{
        return(row["Type_2"])
    }
}

fill_egg_group <- function(row){
    if (row["Egg_Group_2"] == ''){
        return(row["Egg_Group_1"])
    } else{
        return(row["Egg_Group_2"])
    }
}

raw.df$Type_2 <- apply(raw.df, 1, fill_type2)
raw.df$Type_2 <- as.factor(raw.df$Type_2)
raw.df$Egg_Group_2 <- apply(raw.df, 1, fill_egg_group)
raw.df$Egg_Group_2 <- as.factor(raw.df$Egg_Group_2)
head(raw.df)

Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
4,Charmander,Fire,Fire,309,39,52,43,60,50,...,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
5,Charmeleon,Fire,Fire,405,58,64,58,80,65,...,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed
6,Charizard,Fire,Flying,534,78,84,78,109,85,...,Red,True,0.875,Monster,Dragon,True,1.70,90.5,45,bipedal_tailed


In [4]:
colnames(raw.df)

[1] "Number"           "Name"             "Type_1"           "Type_2"          
 [5] "Total"            "HP"               "Attack"           "Defense"         
 [9] "Sp_Atk"           "Sp_Def"           "Speed"            "Generation"      
[13] "isLegendary"      "Color"            "hasGender"        "Pr_Male"         
[17] "Egg_Group_1"      "Egg_Group_2"      "hasMegaEvolution" "Height_m"        
[21] "Weight_kg"        "Catch_Rate"       "Body_Style"

In [5]:
df <- raw.df %>% select(-c(Number, Name, Generation))
df$Type_2 <- as.factor(df$Type_2)
head(df)

Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
Grass,Poison,318,45,49,49,65,65,45,False,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
Grass,Poison,405,60,62,63,80,80,60,False,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
Grass,Poison,525,80,82,83,100,100,80,False,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
Fire,Fire,309,39,52,43,60,50,65,False,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
Fire,Fire,405,58,64,58,80,65,80,False,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed
Fire,Flying,534,78,84,78,109,85,100,False,Red,True,0.875,Monster,Dragon,True,1.70,90.5,45,bipedal_tailed


In [227]:
dist.mat <- daisy(df, metric="gower")

It may occur that two Pokemon share a Type and/or Egg Group in different slots e.g. Water/Ground vs. Ground/Flying. Want to make sure the distance between these two Pokemon accounts for this similarity.

For each pair of Pokemon, check if Type/EG follows this pattern. If so, switch one of the Pokemon's Type/EG (Type_1 -> Type_2, Type_2 -> Type_1) and recalculate distance matrix. Set distance between these two Pokemon the the new value, leave rest unchanged. This is a pretty inefficient way of doing this (recalculating distance matrix each time), but #yolo.

In [286]:
adjust_dual_params <- function(dist.mat, df){
    dist.mat.adj <- as.matrix(dist.mat)

    for ( row1 in 1:(nrow(raw.df)-1) ){
        r1.type1 <- as.character(df[row1,]$Type_1)
        r1.type2 <- as.character(df[row1,]$Type_2)
        r1.eg1 <- as.character(df[row1,]$Egg_Group_1)
        r1.eg2 <- as.character(df[row1,]$Egg_Group_2)

        for ( row2 in (row1+1):(nrow(raw.df)-1) ){ # calculate each pair only once
            r2.type1 <-as.character(df[row2,]$Type_1)
            r2.type2 <-as.character(df[row2,]$Type_2)
            r2.eg1 <- as.character(df[row2,]$Egg_Group_1)
            r2.eg2 <- as.character(df[row2,]$Egg_Group_2)

            # for both type and egg group check that
            #  1) follows pattern
            #  2) dual type/egg group
            type.check <- r1.type1 == r2.type2 | r1.type2 == r2.type1
            type.check <- type.check & (r1.type1 != r1.type2) & (r2.type1 != r2.type2)

            eg.check <- r1.eg1 == r2.eg2 | r1.eg2 == r2.eg1
            eg.check <- eg.check & (r1.eg1 != r1.eg2) & (r2.eg1 != r2.eg2)

            if ( !(type.check | eg.check) ){ # if neither type nor egg group follows pattern
                next
            }

            df.new <- df
            if (type.check){
                df.new[row1,]$Type_1 <- r1.type2
                df.new[row1,]$Type_2 <- r1.type1          
            }
            if (eg.check){
                df.new[row1,]$Egg_Group_1 <- r1.eg2
                df.new[row1,]$Egg_Group_2 <- r1.eg1
            }

            new.gower <- daisy(df.new, metric="gower")
            dist.mat.adj[row1, row2] <- as.matrix(new.gower)[row1, row2]
        }
    }

    return(dist.mat.adj)
}

In [287]:
dist.mat.adj <- adjust_dual_params(dist.mat, df)

In [7]:
write.csv(as.matrix(dist.mat.adj), "../data/processed/pokemon_dm_gower.csv")

In [8]:
df2 <- raw.df %>% select(-c(Number, Name, Generation, Pr_Male, Total, HP, Attack, Defense, Sp_Atk, Sp_Def, Speed, Height_m, Weight_kg, Catch_Rate))
head(df2)

Type_1,Type_2,isLegendary,Color,hasGender,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Body_Style
Grass,Poison,False,Green,True,Monster,Grass,False,quadruped
Grass,Poison,False,Green,True,Monster,Grass,False,quadruped
Grass,Poison,False,Green,True,Monster,Grass,True,quadruped
Fire,Fire,False,Red,True,Monster,Dragon,False,bipedal_tailed
Fire,Fire,False,Red,True,Monster,Dragon,False,bipedal_tailed
Fire,Flying,False,Red,True,Monster,Dragon,True,bipedal_tailed


In [307]:
dist.mat2 <- daisy(df2, metric="gower")

In [308]:
dist.mat2.adj <- adjust_dual_params(dist.mat2, df2)

In [10]:
write.csv(as.matrix(dist.mat2.adj), "../data/processed/pokemon_dm_gower_only_categorical.csv")